<a href="https://colab.research.google.com/github/rutuja-patil24/CMPE-258-Deep_Learning/blob/main/Assignment_06/a)_02_Coding_UseCase_Finetuning_Phi_3_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Step 1: Install Dependencies

In [1]:
!pip install -q unsloth accelerate bitsandbytes datasets peft transformers einops xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

##Step 2: Load Phi-3 Model with Unsloth

In [2]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "microsoft/phi-3-mini-4k-instruct",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)

tokenizer.pad_token = tokenizer.eos_token


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

##Step 3: Enable LoRA Adapters

In [3]:
FastLanguageModel.for_training(model, use_gradient_checkpointing=True)
model = FastLanguageModel.get_peft_model(model)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


##Step 4: Prepare Instruction Dataset

In [9]:
from datasets import Dataset

data = [
    {
        "instruction": "Write a Python function to check if a number is prime.",
        "output": """def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True"""
    },
    {
        "instruction": "Write a Python function to reverse a string.",
        "output": """def reverse_string(s):
    return s[::-1]"""
    }
]

dataset = Dataset.from_list(data)


## Step 5: Format with Chat Template

In [10]:
def format_chat_template(example):
    return {
        "text": f"<|im_start|>system\nYou are a helpful coding assistant.<|im_end|>\n"
                f"<|im_start|>user\n{example['instruction']}\n<|im_end|>\n"
                f"<|im_start|>assistant\n{example['output']}<|im_end|>"
    }

dataset = dataset.map(format_chat_template, batched=False)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

##Step 6: Tokenize Dataset

In [11]:
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

dataset = dataset.map(tokenize, remove_columns=dataset.column_names)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

##Step 7: Training Setup

In [12]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

trainer = Trainer(
    model = model,
    train_dataset = dataset,
    args = TrainingArguments(
        output_dir = "./phi3_codegen_lora",
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        learning_rate = 2e-4,
        num_train_epochs = 3,
        logging_steps = 10,
        fp16 = True,
        save_strategy = "epoch",
        remove_unused_columns = False,
    ),
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
)


##Step 8: Train the Model

In [13]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2 | Num Epochs = 3 | Total steps = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416/4,000,000,000 (0.75% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rutujabhaskarrao (rutujabhaskarrao-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=3, training_loss=0.7839231491088867, metrics={'train_runtime': 29.0181, 'train_samples_per_second': 0.207, 'train_steps_per_second': 0.103, 'total_flos': 69165404651520.0, 'train_loss': 0.7839231491088867, 'epoch': 3.0})

##Step 9: Save LoRA Adapter + Tokenizer

In [14]:
# Save LoRA adapter weights
model.save_pretrained("phi3_codegen_lora_adapter")

# Save tokenizer for reuse during inference
tokenizer.save_pretrained("phi3_codegen_lora_adapter")


('phi3_codegen_lora_adapter/tokenizer_config.json',
 'phi3_codegen_lora_adapter/special_tokens_map.json',
 'phi3_codegen_lora_adapter/tokenizer.model',
 'phi3_codegen_lora_adapter/added_tokens.json',
 'phi3_codegen_lora_adapter/tokenizer.json')

##Step 10: Inference

In [15]:
model.eval()

prompt = """### Instruction:
Write a Python function to calculate the factorial of a number.

### Response:"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Instruction:
Write a Python function to calculate the factorial of a number.

### Response:
Certainly! Below is a Python function that calculates the factorial of a given non-negative integer. The function uses recursion to calculate the factorial.

```python
def factorial(n):
    # Base case: if n is 0 or 1, the factorial is 1
    if n in (0, 1):
        return 1
    # Recursive case: n! = n * (n-1)!

